Displays unweighted plots from the corresponding Coffea outputs.  The plots are categorized according to the histogram name (dependant variable) and the tag category defined in the `TTbarResProcessor`.  
# NOTE: #
All QCD MC histograms are normalized directly to the data, as no corrections are applied via mistag analysis or modmass procedures anyways.  For a more realistic analysis, refer to `TTbarResCoffea_BkgEstAnalysis` notebook.

In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
import hist
from coffea import util
import numpy as np
import itertools
import mplhep as hep

In [ ]:
dir1 = 'CoffeaOutputs/UnweightedOutputs/'
dir2 = 'CoffeaOutputsForCombine/Coffea_FirstRun/'

btagdir = 'MediumBTag/'
yeardir = '2016/'
apvdir = 'noAPV/'

In [ ]:
QCD_unweighted = util.load(dir2+'QCD/'+btagdir+yeardir+apvdir+'TTbarRes_0l_UL16postVFP_QCD.coffea')
TTbar_unweighted = util.load(dir2+'TT/'+btagdir+yeardir+apvdir+'TTbarRes_0l_UL16postVFP_TTbar.coffea')
JetHT2016_unweighted = util.load(dir2+'JetHT/'+btagdir+yeardir+'TTbarRes_0l_JetHT2016_Data.coffea')
RSGluon1000_unweighted = util.load(dir2+'RSGluonToTT/'+btagdir+yeardir+apvdir+'TTbarRes_0l_UL16postVFP_RSGluon1000.coffea')
DM1000_unweighted = util.load(dir2+'ZprimeDMToTTbar/'+btagdir+yeardir+apvdir+'TTbarRes_0l_UL16postVFP_DM1000.coffea')

# The cutflow can be checked if desired 

In [ ]:
Datasets = {
    'QCD': QCD_unweighted,
    'TTbar': TTbar_unweighted,
    'JetHT2016': JetHT2016_unweighted,
    'RSGluon1000': RSGluon1000_unweighted,
    'DM1000': DM1000_unweighted
}

In [ ]:
for dataset,output in Datasets.items():
    print("-------" + dataset + " Cutflow--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %20s' % (i,j) )

## Make Save Directory

In [ ]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

In [ ]:
def DoesDirectoryExist(mypath): #extra precaution (Probably overkill...)
    '''Checks to see if Directory exists before running mkdir_p'''
    import os.path
    from os import path
    
    if path.exists(mypath):
        pass
    else:
        mkdir_p(mypath)

# Prepare to Loop through Analysis Categories and Histograms

In [ ]:
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

# ---- Reiterate categories ---- #
ttagcats = ["AT&Pt", "at", "pret", ">=0t", "0t", ">=1t", "1t", "2t"]
btagcats = ["0b", "1b", "2b"]
ycats = ['cen', 'fwd']

list_of_cats = [ t+b+y for t,b,y in itertools.product( ttagcats, btagcats, ycats) ]
label_dict = {i: label for i, label in enumerate(list_of_cats)}
print(label_dict)

# ---- List the Histograms Here ---- #
list_of_hists_4vector = ('ttbarmass', 'jetpt', 'jeteta', 'jetphi', 'jety', 'jetdy', 'jetmass', 'SDmass', 'tau32')

In [ ]:
maindirectory = os.getcwd() # prepare to locally save images 

# Luminosity, Cross Sections and Scale Factors

In [ ]:
""" ---------------- Luminosity and Cross Sections ---------------- """
Lum     = 137190. # pb^-1 from https://twiki.cern.ch/twiki/bin/viewauth/CMS/PdmVAnalysisSummaryTable
Lum2016 = 35920.

ttbar_BR = 0.45 #Calculated using PDG 2019
ttbar_xs = 831.76  #pb Monte Carlo already includes xs in event weight!!
ttbar_sf2016 = ttbar_xs*Lum2016*ttbar_BR/TTbar_unweighted['cutflow']['sumw'] 

qcd_xs = 1370000000.0 #pb From https://cms-gen-dev.cern.ch/xsdb Set to one as a guess that these MC recieved the same treatment as ttbar
qcd_sf2016 = qcd_xs*Lum2016/QCD_unweighted['cutflow']['sumw']

RSGluon1000_sf2016 = Lum2016/RSGluon1000_unweighted['cutflow']['all events'] # weights are 1 (maybe?)

DM1000_sf2016 = Lum2016/DM1000_unweighted['cutflow']['all events']

In [ ]:
# Nevts2016 = 583876623 #625516390. # from dasgoclient 
# # Nevts2017 = 410461585.
# # Nevts2018 = 676328827.
# # Nevts = Nevts2016 + Nevts2017 + Nevts2018
# Nevts_sf2016 = Nevts2016/JetHT2016_unweighted['cutflow']['all events']
# print(Nevts_sf2016)

# Ratio Plot Definition

In [ ]:
def plotratio(numerator, denominator, ax=None, histtype='errorbar', marker='.', markersize=5., color='k', alpha=0.1):
    NumeratorAxes = numerator.axes
    DenominatorAxes = denominator.axes
    
    # integer number of bins in this axis #
    NumeratorAxis1_BinNumber = NumeratorAxes[0].size - 3 # Subtract 3 to remove overflow
    
    DenominatorAxis1_BinNumber = DenominatorAxes[0].size - 3 
    
    if(NumeratorAxis1_BinNumber != DenominatorAxis1_BinNumber):
        raise Exception('Numerator and Denominator axes are different sizes; Cannot perform division.')
    # else:
    #     Numerator = numerator.to_hist()
    #     Denominator = denominator.to_hist()
        
    ratio = numerator / denominator.values()
    
    if histtype == 'errorbar':
        return hep.histplot(ratio, ax=ax, histtype=histtype, marker=marker, markersize=markersize, color=color)
    elif histtype == 'fill':
        return hep.histplot(ratio, ax=ax, histtype=histtype, color=color, alpha=alpha, lw=5.)
    else:
        return hep.histplot(ratio, ax=ax, histtype=histtype, color=color)

# Data and Unweighted MC Plots
### NOTE that SDmass used axes called jetmass, so code will get confused unless exception is made for SDmass

In [ ]:
# for dataset,output in Datasets.items():
PlotType = 'linear'
for ihist in list_of_hists_4vector:
    SaveDirectory = maindirectory + '/PreliminaryAnalysisPlots/' + ihist + '/' + PlotType + '/' # split histograms into subdirectories
    DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
    for icat, ilabel in label_dict.items(): 
        plt.rcParams.update({
        'font.size': 14,
        'axes.titlesize': 18,
        'axes.labelsize': 18,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12
        })
        fig, (ax, rax) = plt.subplots(
            nrows=2,
            ncols=1,
            figsize=(7,7),
            gridspec_kw={"height_ratios": (3, 1)},
            sharex=True
        )
        fig.subplots_adjust(hspace=.07)
        title = ihist + '  ' + ilabel
        #filename = ihist + '_' + ilabel + '_LinearScale.' + 'png'
        filename = ihist + '_' + ilabel + '.' + 'png'
        
#         ttbar_sf2016 = ttbar_xs*Lum2016*ttbar_BR/TTbar_unweighted['cutflow'][icat]
#         qcd_sf2016 = qcd_xs*Lum2016/QCD_unweighted['cutflow'][icat]
#         RSGluon1000_sf2016 = RSGluon1000_xs*Lum2016/RSGluon1000_unweighted['cutflow'][icat]

        
#    ===================================================================================================================
#    DDDD       A    TTTTTTT    A        H     H IIIIIII   SSSSS TTTTTTT   OOO   GGGGGGG RRRRRR     A    M     M   SSSSS     
#    D   D     A A      T      A A       H     H    I     S         T     O   O  G       R     R   A A   MM   MM  S          
#    D    D   A   A     T     A   A      H     H    I    S          T    O     O G       R     R  A   A  M M M M S           
#    D     D  AAAAA     T     AAAAA      HHHHHHH    I     SSSSS     T    O     O G  GGGG RRRRRR   AAAAA  M  M  M  SSSSS      
#    D    D  A     A    T    A     A     H     H    I          S    T    O     O G     G R   R   A     A M     M       S     
#    D   D   A     A    T    A     A     H     H    I         S     T     O   O  G     G R    R  A     A M     M      S      
#    DDDD    A     A    T    A     A     H     H IIIIIII SSSSS      T      OOO    GGGGG  R     R A     A M     M SSSSS
#    ===================================================================================================================
        
        Data_hist = JetHT2016_unweighted[ihist]['JetHT2016_Data', icat, :]
        # Data_hist *= Nevts_sf2016 #scale according to number of events in dataset
        
#    ===================================================================================================
#    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE       QQQ     CCCC  DDDD        
#    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E            Q   Q   C      D   D       
#    N N   N O     O R     R M M M M  A   A  L          I        Z   E           Q     Q C       D    D      
#    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE     Q     Q C       D     D     
#    N   N N O     O R   R   M     M A     A L          I      Z     E           Q   Q Q C       D    D      
#    N    NN  O   O  R    R  M     M A     A L          I     Z      E            Q   Q   C      D   D       
#    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE       QQQ Q   CCCC  DDDD
#    ===================================================================================================
        
        QCD_hist = QCD_unweighted[ihist]['UL16postVFP_QCD', icat, :]
        QCD_hist *= qcd_sf2016 #scaled according to luminosity

#    ===================================================================================================================
#    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE     TTTTTTT TTTTTTT BBBBBB     A    RRRRRR      
#    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E              T       T    B     B   A A   R     R     
#    N N   N O     O R     R M M M M  A   A  L          I        Z   E              T       T    B     B  A   A  R     R     
#    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE        T       T    BBBBBB   AAAAA  RRRRRR      
#    N   N N O     O R   R   M     M A     A L          I      Z     E              T       T    B     B A     A R   R       
#    N    NN  O   O  R    R  M     M A     A L          I     Z      E              T       T    B     B A     A R    R      
#    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE        T       T    BBBBBB  A     A R     R 
#    ===================================================================================================================
        
        TTbar_hist = TTbar_unweighted[ihist]['UL16postVFP_TTbar', icat, :]
        TTbar_hist *= ttbar_sf2016 #scaled according to luminosity

#    ===================================================================================================================================
#    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE     RRRRRR    SSSSS GGGGGGG L       U     U   OOO   N     N     
#    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E           R     R  S      G       L       U     U  O   O  NN    N     
#    N N   N O     O R     R M M M M  A   A  L          I        Z   E           R     R S       G       L       U     U O     O N N   N     
#    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE     RRRRRR   SSSSS  G  GGGG L       U     U O     O N  N  N     
#    N   N N O     O R   R   M     M A     A L          I      Z     E           R   R         S G     G L       U     U O     O N   N N     
#    N    NN  O   O  R    R  M     M A     A L          I     Z      E           R    R       S  G     G L        U   U   O   O  N    NN     
#    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE     R     R SSSSS    GGGGG  LLLLLLL   UUU     OOO   N     N 
#    ===================================================================================================================================

        RSG1000_hist = RSGluon1000_unweighted[ihist]['UL16postVFP_RSGluon1000', icat, :]
        RSG1000_hist *= RSGluon1000_sf2016 #scaled according to luminosity  
        
#    ===========================================================================================
#    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE     DDDD    M     M     
#    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E           D   D   MM   MM     
#    N N   N O     O R     R M M M M  A   A  L          I        Z   E           D    D  M M M M     
#    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE     D     D M  M  M     
#    N   N N O     O R   R   M     M A     A L          I      Z     E           D    D  M     M     
#    N    NN  O   O  R    R  M     M A     A L          I     Z      E           D   D   M     M     
#    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE     DDDD    M     M
#    ===========================================================================================
        
        DM1000_hist = DM1000_unweighted[ihist]['UL16postVFP_DM1000', icat, :]
        DM1000_hist *= DM1000_sf2016 #scaled according to luminosity
            
#    ===========================================================================
#    M     M    A    K     K EEEEEEE     PPPPPP  L         OOO   TTTTTTT   SSSSS     
#    MM   MM   A A   K   K   E           P     P L        O   O     T     S          
#    M M M M  A   A  K K     E           P     P L       O     O    T    S           
#    M  M  M  AAAAA  KKk     EEEEEEE     PPPPPP  L       O     O    T     SSSSS      
#    M     M A     A K  K    E           P       L       O     O    T          S     
#    M     M A     A K   K   E           P       L        O   O     T         S      
#    M     M A     A K   K   EEEEEEE     P       LLLLLLL   OOO      T    SSSSS 
#    ===========================================================================

        MC_hist = TTbar_hist.copy()
        MC_hist += QCD_hist
        
        # ---- Extract both the data and MC events and from histograms ---- #
        NtotalMC = np.sum(MC_hist.view().value)
        NtotalData = np.sum(Data_hist.view().value)
        
        # ---- Normalize the total MC histogram directly to the data (for aesthetic purposes only!) ---- #
        # -------- Unweighted simulation of the background alone greatly overestimates ------- #
        if NtotalMC > 0.:
            MC_hist *= (NtotalData/NtotalMC)
            TTbar_hist *= (NtotalData/NtotalMC)
        else:
            MC_hist *= 0.
            TTbar_hist *= 0.

        #---- Plot Data ----#
        #-----------------------------------------------------------------#
        Data_hist.plot1d(ax=ax, histtype='errorbar', marker='.', markersize=5., color='k')
        
        #---- Plot Total MC (simulated QCD + SM ttbar background)----#
        #-----------------------------------------------------------------#
        MC_hist.plot1d(ax=ax, histtype='fill', color='yellow')
        
        #---- Plot TTbar MC for comparison ---- #
        #-----------------------------------------------------------------#    
        TTbar_hist.plot1d(ax=ax, histtype='fill', color='red')
        
        #---- Plot RSG MC for comparison ---- #
        #-----------------------------------------------------------------#   
        RSG1000_hist.plot1d(ax=ax, histtype='step', color='purple')
        
        
        #---- Plot DMM MC for comparison ----#
        #-----------------------------------------------------------------#
        DM1000_hist.plot1d(ax=ax, histtype='step', color='black')
        
        ax.set_ylim(bottom=0.1)
        ax.set_yscale(PlotType)
        ax.set_ylabel('Events')
        ax.set_xlabel(None)
        ax.set_title(title)
        leg = ax.legend(labels=[r'QCD Sim. Bkg', r'$t\bar{t}$ Sim.', r'RSKK Gluon $1TeV$ Sim.', r'DM Med. $1TeV$ Sim.', r'Data'], fontsize='xx-small')
        
        
        #---- Plot Ratio ----#
        plotratio(Data_hist, MC_hist, ax = rax, histtype = 'errorbar')
        rax.set_ylabel('Data/MC')
        rax.axhline(y=1, color='k', linestyle=':')
        rax.set_ylim(0,2)
        
        if ihist == 'ttbarmass':
            rax.set_xlim(700,5000)
        if ihist == 'jetpt':
            rax.set_xlim(400,500)
        if ihist == 'jeteta':
            rax.set_xlim(-3,3)
        if ihist == 'tau32':
            rax.set_xlim(0,1.2)

        #---- Labeling ----#
        Lint = str(Lum2016*.001) # Integrated Luminosity
        lumi = plt.text(1.15, 1.07, Lint[:6] + " fb$^{-1}$",
                fontsize=16,
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes
               )
#         plt.savefig(SaveDirectory+filename, bbox_inches="tight")
        print(SaveDirectory+filename)